## 1. import

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import faiss
import pickle
import numpy as np
from langchain.schema import Document
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoModel, AutoTokenizer
import torch

def get_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")
    # 모든 입력 텐서를 GPU로 이동
    inputs = {key: val.to(device) for key, val in inputs.items()}
    pool_mask = torch.ones(inputs["input_ids"].shape, dtype=torch.long).to(device)

    with torch.no_grad():
        outputs = model(**inputs, pool_mask=pool_mask)

    return outputs.embedding.cpu().numpy()  # GPU에서 CPU로 변환 후 반환

model_name = "kakaocorp/kanana-nano-2.1b-embedding"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

c:\Users\USER\anaconda3\envs\vectordb_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [36]:
# 쿼리문 (50여개) 정의
querys = [
"하나와 롯데 자동차 보험의 어린이 보호구역 사고 시 과실 인정 기준 차이는?",
"하나손해보험과 롯데손해보험의 교통사고 처리 지원금 보장 횟수와 대상 범위 차이는?",
"하나손해보험과 롯데손해보험의 예방접종 후 부작용 보장 여부 비교해줘.",
"롯데손해보험과 하나손해보험의 항공기 지연 시 보장 금액과 지연 시간 기준 비교해줘.",
"하나손해보험과 롯데손해보험의 마일리지 할인 제도 적용 기준과 유지 조건은?",
"하나손해보험과 롯데손해보험의 암 보험 약관에서 갑상선암을 일반암으로 인정하는 기준이 있는 쪽은 어디야?",
"어떤 상해보험을 가입해야 골절 진단 받을때 보장받을 수 있어?",
"암 재발하면 진단금 또 받을 수 있어요?",
"보험금 타먹는 꿀팁 좀 알려줘.",
"해외에서 여권을 분실했지만, 재외공관에 신고하지 않고 여행사 도움으로 임시증명서를 받아 귀국했습니다. 이 경우에도 추가 체류비용 보상을 받을 수 있나요?",
"여권 분실로 귀국이 지연되어 3일을 더 머물렀습니다. 추가 체류비용을 모두 청구할 수 있나요?",
"가족력 있어도 암 보험 가입에 제약조건이 있나요?",
"회사 계단에서 넘어져 발목 인대를 다쳤는데, 통원 치료만 받은 경우에도 상해보험에서 치료비가 지급될 수 있어?",
"기존에 당뇨가 있는 경우에도 입원비나 진단비를 받을 수 있나요?",
"암 진단 이후 직장을 쉬면서 치료받고 있는데, 소득보장보험처럼 생활비 보장도 받을 수 있는 특약이 있을까?",
"대장 용종 제거했는데 조직검사 결과가 초기암이래. 이런 경우 암 진단비 받을 수 있어? 얼마나 받을 수 있어?"
]

## 2. DB 불러오기

In [37]:
model_name = "kakaocorp/kanana-nano-2.1b-embedding"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def get_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")
    # 모든 입력 텐서를 GPU로 이동
    inputs = {key: val.to(device) for key, val in inputs.items()}
    pool_mask = torch.ones(inputs["input_ids"].shape, dtype=torch.long).to(device)

    with torch.no_grad():
        outputs = model(**inputs, pool_mask=pool_mask)

    return outputs.embedding.cpu().numpy()

In [38]:
# 하나손해보험
# 1️⃣ 저장된 FAISS 인덱스 로드
def hana_faiss(query):
    index = faiss.read_index("./faiss_hana/faiss_index_hana.bin")

    # 2️⃣ 문서 정보 로드
    with open("./faiss_hana/documents_hana.pkl", "rb") as f:
        documents = pickle.load(f)

    # 3️⃣ 검색 수행
    query_embedding = get_embeddings([query])[0]  # 쿼리 임베딩
    query_embedding = np.array([query_embedding], dtype=np.float32)

    D, I = index.search(query_embedding, k=10)  # 가장 유사한 5개 검색
    context_hana = []

    # 4️⃣ 검색 결과 출력
    for idx in I[0]:
        context_hana.append(documents[idx].page_content)
        
    return context_hana


In [40]:
# 롯데손해보험
# 1️⃣ 저장된 FAISS 인덱스 로드
def lotte_faiss(query):
    index = faiss.read_index("./faiss_lotte/faiss_index_lotte.bin")

    # 2️⃣ 문서 정보 로드
    with open("./faiss_lotte/documents_lotte.pkl", "rb") as f:
        documents = pickle.load(f)

    # 3️⃣ 검색 수행
    query_embedding = get_embeddings([query])[0]  # 쿼리 임베딩
    query_embedding = np.array([query_embedding], dtype=np.float32)

    D, I = index.search(query_embedding, k=10)  # 가장 유사한 5개 검색
    context_lotte = []

    # 4️⃣ 검색 결과 출력
    for idx in I[0]:
        context_lotte.append(documents[idx].page_content)
        return context_lotte

In [41]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

qna_test = {}
system_message = """
당신은 보험 상담 ASSISTANT입니다. 
각 보험사의 약관인 context 들을 기반으로 query에 대해 답변해 주세요.
HANA_context : 하나손해보험사의 보험 상품 약관
LOTTE_context : 롯데손해보험사의 보험 상품 약관
"""

# ChatPromptTemplate에서 사용할 템플릿 정의
prompt_template = ChatPromptTemplate.from_template(
    system_message + "\n사용자 질문: {query}\nHANA_context: {HANA_context}\nLOTTE_context: {LOTTE_context}"
)



for i, query in enumerate(querys):
    context_lotte = lotte_faiss(query)
    context_hana = hana_faiss(query)

    # 템플릿에 실제 값을 채워 메시지 생성
    messages = prompt_template.format_messages(
        query=query, 
        HANA_context=context_hana, 
        LOTTE_context=context_lotte
    )

    answer = ChatOpenAI(model_name='gpt-4o').invoke(messages)  # messages.to_messages()를 사용할 필요 없음

    qna_test[f'question_{i}'] = query
    qna_test[f'answer_{i}'] = answer.content  # answer가 Message 객체이므로 content 추출

print(qna_test)


{'question_0': '하나와 롯데 자동차 보험의 어린이 보호구역 사고 시 과실 인정 기준 차이는?', 'answer_0': '하나손해보험과 롯데손해보험의 약관에서 어린이 보호구역 사고 시 과실 인정 기준에 대한 구체적인 정보는 주어지지 않았습니다. \n\n하지만 일반적인 자동차 보험의 경우, 어린이 보호구역 내에서 사고가 발생하면 운전자에게 보다 엄격한 과실 기준이 적용될 수 있으며, 이는 보험사마다 다르게 적용될 수 있습니다. 일반적으로 어린이 보호구역에서는 운전자가 주의 의무를 더욱 철저히 해야 하며, 사고 발생 시 운전자에게 더 큰 책임이 부과될 가능성이 높습니다. \n\n정확한 정보를 얻기 위해서는 각 보험사에 직접 문의하거나, 해당 보험사의 자동차 보험 약관을 자세히 살펴보시는 것이 좋습니다.', 'question_1': '하나손해보험과 롯데손해보험의 교통사고 처리 지원금 보장 횟수와 대상 범위 차이는?', 'answer_1': '현재 제공된 context 정보에서는 하나손해보험과 롯데손해보험의 교통사고 처리 지원금에 대한 보장 횟수와 대상 범위에 대한 구체적인 차이를 확인할 수 없습니다. \n\nHANA_context와 LOTTE_context에 제공된 정보는 주로 보험 상품에 대한 일반적인 안내, 예금자 보호에 관한 정보, 사고 접수 및 보험 처리 관련 안내 등이 포함되어 있으며, 교통사고 처리 지원금에 대한 구체적인 보장 횟수나 대상 범위에 대한 정보는 명시되어 있지 않습니다.\n\n보험 약관의 세부 내용은 각 보험사의 공식 웹사이트나 고객센터를 통해 확인하실 수 있으며, 필요시 직접 보험사에 문의하시는 것이 정확한 정보를 얻는 방법입니다. 하나손해보험의 고객센터 전화번호는 1566-3000, 롯데손해보험의 고객센터 전화번호는 1588-3344 또는 1600-3434입니다.', 'question_2': '하나손해보험과 롯데손해보험의 예방접종 후 부작용 보장 여부 비교해줘.', 'answer_2': '하나손해보험과 롯데손해보험의 약관 내용에

In [43]:
json_list = []
for i in range(len(querys)):
    tmp_dict={}
    tmp_dict['question'] = qna_test[f'question_{i}']
    tmp_dict['answer'] = qna_test[f'answer_{i}']
    json_list.append(tmp_dict)

In [ ]:
import json
file_path = 'finetunning_tmp.json'

with open(file_path, 'w', encoding='UTF-8-sig') as file:
     file.write(json.dumps(json_list, ensure_ascii=False, indent=3))